In [1]:
import pandas as pd
import dateutil
import numpy as np
import re
import json

In [2]:
current_prod=pd.read_csv(
    "Chicago_Police_Department_Officers.csv",
parse_dates=["employment date"], dtype={"birth year": "Int64", "badge number": "Int64"}
).set_index("id", drop=False)

In [3]:
current_prod["middle initial"] = current_prod["middle initial"].fillna("").str.strip()

In [4]:
current_prod["last name"] = current_prod["last name"].str.strip()

In [5]:
current_prod["first name"] = current_prod["first name"].str.strip()

In [6]:
current_prod.head()

,id,unique identifier,last name,first name,middle initial,suffix,gender,race,birth year,employment date,badge number,job title,most recent salary
id,,,,,,,,,,,,,
17511,17511,NaN,KLICH,ROBERT,H,NaN,M,WHITE,1957,1982-08-30,5,COMMANDER,NaN
17750,17750,NaN,LAJEWSKI,ROBERT,J,NaN,M,WHITE,1959,1985-12-16,23,CAPTAIN OF POLICE,NaN
17749,17749,NaN,LAIDLAW,CRAIG,T,NaN,M,WHITE,1983,2013-03-05,17695,POLICE OFFICER,NaN
17748,17748,NaN,LAI,HENRY,H,NaN,M,ASIAN/PACIFIC ISLANDER,1971,2010-09-01,16486,POLICE OFFICER,NaN
17747,17747,NaN,LAHORI,JOHN,K,NaN,M,BLACK,1953,1990-07-30,9852,POLICE OFFICER,NaN


In [7]:
current_prod.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11941 entries, 17511 to 11942
Data columns (total 13 columns):
id                    11941 non-null int64
unique identifier     0 non-null float64
last name             11941 non-null object
first name            11941 non-null object
middle initial        11941 non-null object
suffix                0 non-null float64
gender                11941 non-null object
race                  11941 non-null object
birth year            9699 non-null Int64
employment date       11941 non-null datetime64[ns]
badge number          11755 non-null Int64
job title             11941 non-null object
most recent salary    0 non-null float64
dtypes: Int64(2), datetime64[ns](1), float64(3), int64(1), object(6)
memory usage: 1.3+ MB


In [8]:
cpdp_profiles = pd.read_csv("final-profiles.csv",
                            dtype={4:"str", 15: "str", "birth_year": "Int64", "link_UID": "Int64",
                                  "current_status": "Int64", "current_star": "Int64",
                                  "current_unit": "Int64"},
                            parse_dates=["appointed_date", "resignation_date", "start_date", "org_hire_date"]
                           ).set_index("UID", drop=False)

In [9]:
cpdp_profiles[["first_name", "last_name", "middle_initial", "middle_initial2", "suffix_name",
               "current_rank", "cleaned_rank"]]=cpdp_profiles[["first_name", "last_name", "middle_initial", "middle_initial2", "suffix_name",
               "current_rank", "cleaned_rank"]].fillna("")

In [10]:
cpdp_profiles = cpdp_profiles.loc[:133667].copy() # tail end does not match cpdp online data

In [11]:
cpdp_profiles["middle_initial"]=cpdp_profiles["middle_initial"].fillna("").str.strip()

In [12]:
cpdp_profiles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33667 entries, 100001 to 133667
Data columns (total 20 columns):
UID                 33667 non-null int64
first_name          33667 non-null object
last_name           33667 non-null object
middle_initial      33667 non-null object
middle_initial2     33667 non-null object
suffix_name         33667 non-null object
birth_year          32562 non-null Int64
race                33477 non-null object
gender              33643 non-null object
appointed_date      33594 non-null datetime64[ns]
resignation_date    20194 non-null datetime64[ns]
current_status      31789 non-null Int64
current_star        15364 non-null Int64
current_unit        31606 non-null Int64
current_rank        33667 non-null object
start_date          18971 non-null datetime64[ns]
org_hire_date       19786 non-null datetime64[ns]
profile_count       33667 non-null int64
cleaned_rank        33667 non-null object
link_UID            33667 non-null Int64
dtypes: Int64(5), da

In [13]:
import re
def last_name_key(name):
    if name:
        n = re.sub(r"[^A-Z ]","",name.upper())
        pieces = n.split()
        if pieces[-1] in ["JR", "SR", "I", "II", "III", "IV"]:
            pieces = pieces[:-1]
        return "".join(pieces)
    return ""

In [14]:
cpdp_profiles["last_name_key"] = cpdp_profiles["last_name"].apply(last_name_key)

In [15]:
current_prod["last_name_key"]=current_prod["last name"].apply(last_name_key)

In [16]:
def get_match_weight_vector(pool, series, entry):
    if isinstance(entry, tuple):
        left, right, weight = entry
    else:
        raise Exception("Invalid key")
    res = (pool[right]==series[left]).astype("int")
    return res*weight

def match_number_from_keys(pool, series, keys):
    selection = None
    for entry in keys:
        selection_this_key=get_match_weight_vector(pool, series, entry)
        if selection is None:
            selection = selection_this_key
        else:
            selection += selection_this_key
    return selection
    
    
def attempt_merge(target, pool, min_match, keys, additional_keys=None):
    """
    Attempt to find a match in pool for each row in target
    """
    missed = []
    matched = []
    for idx, series in target.iterrows():
        matched_fields = match_number_from_keys(pool, series, keys)
        add_match_factor = match_number_from_keys(pool, series, additional_keys) / len(additional_keys)
        best_match=matched_fields.nlargest(n=2)
        match_number = best_match.max()
        if match_number < min_match:
            print(f"No match for {idx} - best match got {match_number}")
            missed += [(idx, [])]
            continue
        if match_number>best_match.min():
            
            match_idx = best_match.index[0]
            matched+=[{"left_idx": idx, "right_idx": match_idx,
                       "score": match_number+add_match_factor.loc[match_idx]}]
            continue
        print(f"Could not resolve multiple matches for {idx} - score: {match_number}")
        missed += [(idx, list(matched_fields[matched_fields>=match_number].index))]

    return {"matched": matched, "missed": missed}

## merge
Attempt merge between OpenOversight production data and cpdp officer data, using weights on the different attributes

In [17]:
merge_result=attempt_merge(current_prod, cpdp_profiles,5,
              [("first name","first_name",3),("last_name_key","last_name_key",3),
               ("employment date", "appointed_date",2),
             ("birth year", "birth_year",1),("badge number", "current_star",1),("middle initial", "middle_initial",1)],
             [("gender","gender",1),
              ("race", "race",1)])

Could not resolve multiple matches for 22151 - score: 9
Could not resolve multiple matches for 22723 - score: 9
Could not resolve multiple matches for 22785 - score: 10
Could not resolve multiple matches for 12472 - score: 10


In [18]:
merge_df = pd.DataFrame.from_dict(merge_result["matched"])

In [19]:
right_counts = merge_df.groupby("right_idx").size()
right_counts[right_counts>1]

right_idx
116946    2
dtype: int64

In [20]:
right_counts[right_counts>1].index

Int64Index([116946], dtype='int64', name='right_idx')

In [21]:
merge_df[merge_df["right_idx"].isin(right_counts[right_counts>1].index)]["left_idx"]

9915    13964
9916    13963
Name: left_idx, dtype: int64

In [22]:
double_values = list(merge_df[merge_df["right_idx"].isin(right_counts[right_counts>1].index)]["left_idx"])

In [23]:
missed_first_run = current_prod.loc[list([a for a,b in merge_result["missed"]]+double_values)]

In [24]:
missed_first_run

,id,unique identifier,last name,first name,middle initial,suffix,gender,race,birth year,employment date,badge number,job title,most recent salary,last_name_key
id,,,,,,,,,,,,,,
22151,22151,NaN,SPARANO,FREDERICK,K,NaN,M,WHITE,1985,1985-11-18,21233,PO AS DETECTIVE,NaN,SPARANO
22723,22723,NaN,TORRES,RICARDO,,NaN,M,WHITE HISPANIC,NaN,2014-05-27,16756,POLICE OFFICER,NaN,TORRES
22785,22785,NaN,TRIPOLI,JOSEPH,A,NaN,M,WHITE,1961,1994-07-05,20571,PO AS DETECTIVE,NaN,TRIPOLI
12472,12472,NaN,BANSLEY,JAMES,A,NaN,M,WHITE,1983,2009-12-16,8791,POLICE OFFICER,NaN,BANSLEY
13964,13964,NaN,CONNORS,KEVIN,M,NaN,M,WHITE,1975,1999-09-13,1182,SERGEANT OF POLICE,NaN,CONNORS
13963,13963,NaN,CONNORS,KEVIN,M,NaN,M,WHITE,1975,1999-09-13,1182,SERGEANT OF POLICE,NaN,CONNORS


In [25]:
merge_df=merge_df.drop(merge_df[merge_df["right_idx"].isin(right_counts[right_counts>1].index)].index)

In [26]:
missed_first_run

,id,unique identifier,last name,first name,middle initial,suffix,gender,race,birth year,employment date,badge number,job title,most recent salary,last_name_key
id,,,,,,,,,,,,,,
22151,22151,NaN,SPARANO,FREDERICK,K,NaN,M,WHITE,1985,1985-11-18,21233,PO AS DETECTIVE,NaN,SPARANO
22723,22723,NaN,TORRES,RICARDO,,NaN,M,WHITE HISPANIC,NaN,2014-05-27,16756,POLICE OFFICER,NaN,TORRES
22785,22785,NaN,TRIPOLI,JOSEPH,A,NaN,M,WHITE,1961,1994-07-05,20571,PO AS DETECTIVE,NaN,TRIPOLI
12472,12472,NaN,BANSLEY,JAMES,A,NaN,M,WHITE,1983,2009-12-16,8791,POLICE OFFICER,NaN,BANSLEY
13964,13964,NaN,CONNORS,KEVIN,M,NaN,M,WHITE,1975,1999-09-13,1182,SERGEANT OF POLICE,NaN,CONNORS
13963,13963,NaN,CONNORS,KEVIN,M,NaN,M,WHITE,1975,1999-09-13,1182,SERGEANT OF POLICE,NaN,CONNORS


In [27]:
remaining_profiles=cpdp_profiles.drop(merge_df["right_idx"])

In [28]:
merge_result_second=attempt_merge(missed_first_run, remaining_profiles,5,
              [("first name","first_name",3),("last_name_key","last_name_key",3),
               ("employment date", "appointed_date",2),
             ("birth year", "birth_year",1),("badge number", "current_star",1),("middle initial", "middle_initial",1)],
             [("gender","gender",1),
              ("race", "race",1)])

Could not resolve multiple matches for 22785 - score: 10


In [29]:
merge_result_second["missed"]

[(22785, [115847, 115848])]

In [30]:
merge_result_second["matched"]

[{'left_idx': 22151, 'right_idx': 109110, 'score': 9.5},
 {'left_idx': 22723, 'right_idx': 124059, 'score': 9.0},
 {'left_idx': 12472, 'right_idx': 111205, 'score': 10.5},
 {'left_idx': 13964, 'right_idx': 116946, 'score': 11.5},
 {'left_idx': 13963, 'right_idx': 116946, 'score': 11.5}]

In [31]:
right_counts = pd.DataFrame.from_dict(merge_result_second["matched"]+merge_result["matched"]).groupby("right_idx").size()
right_counts[right_counts>1]

right_idx
116946    4
dtype: int64

In [32]:
cpdp_profiles.loc[[116946, 116945]]

,UID,first_name,last_name,middle_initial,middle_initial2,suffix_name,birth_year,race,gender,appointed_date,...,current_status,current_star,current_unit,current_rank,start_date,org_hire_date,profile_count,cleaned_rank,link_UID,last_name_key
UID,,,,,,,,,,,,,,,,,,,,,
116946,116946,KEVIN,CONNORS,M,,,1975,WHITE,MALE,1999-09-13,...,1,1182,8,SERGEANT OF POLICE,1999-09-13,1999-09-13,13,SERGEANT,5204,CONNORS
116945,116945,KEVIN,CONNORS,M,,,1967,WHITE,MALE,1999-09-13,...,1,15151,7,POLICE OFFICER,1999-09-13,1999-09-13,10,POLICE OFFICER,5205,CONNORS


In [33]:
current_prod.loc[[13963, 13964]]

,id,unique identifier,last name,first name,middle initial,suffix,gender,race,birth year,employment date,badge number,job title,most recent salary,last_name_key
id,,,,,,,,,,,,,,
13963,13963,NaN,CONNORS,KEVIN,M,NaN,M,WHITE,1975,1999-09-13,1182,SERGEANT OF POLICE,NaN,CONNORS
13964,13964,NaN,CONNORS,KEVIN,M,NaN,M,WHITE,1975,1999-09-13,1182,SERGEANT OF POLICE,NaN,CONNORS


In [34]:
manually_assigned = [
    {"left_idx": 22785, "right_idx": 115848},
    {"left_idx": 13963, "right_idx": 116945},
    {"left_idx": 13964, "right_idx": 116946}
]

In [35]:
all_matched = merge_result["matched"]+merge_result_second["matched"]+manually_assigned

In [36]:
all_matched_df = pd.DataFrame.from_dict(all_matched).groupby("right_idx").last().reset_index()

In [37]:
all_matched_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11941 entries, 0 to 11940
Data columns (total 3 columns):
right_idx    11941 non-null int64
left_idx     11941 non-null int64
score        11939 non-null float64
dtypes: float64(1), int64(2)
memory usage: 279.9 KB


In [38]:
all_matched_df["left_idx"].is_unique

True

In [39]:
all_matched_df["right_idx"].is_unique

True

# merge & validate

In [40]:
all_merge = current_prod.merge(
    all_matched_df, left_index=True, right_on="left_idx", how="left"
).join(
    cpdp_profiles, how="left", on="right_idx", lsuffix="_oos", rsuffix="_cpdp"
)

In [41]:
all_merge

,id,unique identifier,last name,first name,middle initial,suffix,gender_oos,race_oos,birth year,employment date,...,current_status,current_star,current_unit,current_rank,start_date,org_hire_date,profile_count,cleaned_rank,link_UID,last_name_key_cpdp
9124,17511,NaN,KLICH,ROBERT,H,NaN,M,WHITE,1957,1982-08-30,...,1,5,1,COMMANDER,1982-08-30,1982-08-30,9,COMMANDER,14808,KLICH
9130,17750,NaN,LAJEWSKI,ROBERT,J,NaN,M,WHITE,1959,1985-12-16,...,0,23,26,CAPTAIN OF POLICE,1985-12-16,1985-12-16,10,CAPTAIN,15569,LAJEWSKI
1847,17749,NaN,LAIDLAW,CRAIG,T,NaN,M,WHITE,1983,2013-03-05,...,1,17695,3,POLICE OFFICER,2013-03-05,2013-03-05,9,POLICE OFFICER,15568,LAIDLAW
3829,17748,NaN,LAI,HENRY,H,NaN,M,ASIAN/PACIFIC ISLANDER,1971,2010-09-01,...,1,10025,24,POLICE OFFICER,2010-09-01,2010-09-01,12,POLICE OFFICER,15567,LAI
4893,17747,NaN,LAHORI,JOHN,K,NaN,M,BLACK,1953,1990-07-30,...,0,9852,51,POLICE OFFICER,1990-07-30,1990-07-30,9,POLICE OFFICER,15565,LAHORI
9607,17746,NaN,LAGUNAS,SAMUEL,,NaN,M,WHITE HISPANIC,NaN,2005-09-26,...,1,19202,124,POLICE OFFICER,2005-09-26,2005-09-26,12,POLICE OFFICER,15561,LAGUNAS
173,17745,NaN,LAGUNAS,ALEJANDRO,,NaN,M,WHITE HISPANIC,NaN,2002-10-28,...,1,9916,11,POLICE OFFICER,2002-10-28,2002-10-28,12,POLICE OFFICER,15562,LAGUNAS
608,17744,NaN,LAFATA,ANTHONY,B,NaN,M,WHITE,1967,1995-06-05,...,1,7176,16,POLICE OFFICER,1995-06-05,1995-06-05,11,POLICE OFFICER,15553,LAFATA
6835,17743,NaN,LADUZINSKY,MARK,J,NaN,M,WHITE,1966,1995-01-03,...,1,17231,261,POLICE OFFICER,1995-01-03,1995-01-03,12,POLICE OFFICER,15552,LADUZINSKY
9525,17742,NaN,LADD,RYAN,M,NaN,M,WHITE,1977,2001-05-29,...,1,11878,50,POLICE OFFICER,2001-05-29,2001-05-29,12,POLICE OFFICER,15546,LADD


In [42]:
len(all_merge) == len(current_prod)

True

In [43]:
import jellyfish
def comp_custom(func):
    def comp_cust(x):
        a = x.iloc[0]
        b = x.iloc[1]
        if pd.isna(a):
            if pd.isna(b):
                return -2
            else:
                return -1
        elif pd.isna(b):
            return -1
        else:
            return func(a,b)
    return comp_cust

def comp_obj(x):
    return comp_custom(lambda a,b: 1-int(a==b))(x)

def comp_str(x):
    return comp_custom(jellyfish.levenshtein_distance)(x)

In [44]:
all_merge["first_name_comp"]=all_merge[["first name", "first_name"]].apply(comp_str, axis=1)

In [45]:
all_merge["last_name_comp"]=all_merge[["last_name_key_oos", "last_name_key_cpdp"]].apply(comp_str, axis=1)

In [46]:
all_merge["middle_initial_comp"]=all_merge[["middle initial", "middle_initial"]].apply(comp_str, axis=1)

In [47]:
all_merge["gender_comp"]=all_merge[["gender_oos", "gender_cpdp"]].apply(
    comp_custom(lambda a,b: int(a!=b[0])), axis=1)

In [48]:
all_merge["race_comp"]=all_merge[["race_oos", "race_cpdp"]].apply(
    comp_custom(lambda a,b: 1-int(a.endswith(b) or ("NATIVE" in a and "NATIVE" in b))), axis=1)

In [49]:
all_merge["birth_year_comp"]=all_merge[["birth year", "birth_year"]].apply(comp_obj, axis=1)

In [50]:
all_merge["rank_comp"]=all_merge[["job title", "current_rank"]].apply(comp_str, axis=1)

In [51]:
all_merge["employment_date_comp"]=all_merge[["employment date", "appointed_date"]].apply(comp_obj, axis=1)

In [52]:
all_merge["badge_number_comp"]=all_merge[["badge number", "current_star"]].apply(comp_obj, axis=1)

In [53]:
rel_columns=["id", "last name", "last_name","last_name_key_oos",
             "first name","first_name","first_name_comp",
             "middle initial", "middle_initial","middle_initial2","middle_initial_comp",
             "gender_oos","gender_cpdp", "gender_comp",
             "race_oos","race_cpdp","race_comp",
            "birth year", "birth_year", "birth_year_comp",
             "job title","current_rank","cleaned_rank",
             "employment date", "appointed_date","start_date", "org_hire_date","employment_date_comp",
             "badge number", "current_star","badge_number_comp",
             "UID",  
             "suffix_name", 
            "resignation_date",   
            "last_name_key_cpdp", "last_name_comp"]

In [54]:
all_merge[rel_columns][all_merge["last_name_comp"]>=3]

,id,last name,last_name,last_name_key_oos,first name,first_name,first_name_comp,middle initial,middle_initial,middle_initial2,...,org_hire_date,employment_date_comp,badge number,current_star,badge_number_comp,UID,suffix_name,resignation_date,last_name_key_cpdp,last_name_comp
2166,17277,KAPPEL DONEGAN,KAPPEL,KAPPELDONEGAN,DANIELLE,DANIELLE,0,M,M,,...,2000-08-14,0,18761,18761,0,105247,,NaT,KAPPEL,7
6712,16681,HORAN,HORAN-DAWSON,HORAN,MARIANNE,MARIANNE,0,,,,...,1999-03-08,0,5777,5777,0,118941,,NaT,HORANDAWSON,6
4015,18451,MARSHALL,BAILEY,MARSHALL,JADA,JADA,0,M,M,,...,2006-07-16,0,4997,4997,0,111115,,NaT,BAILEY,7
9874,18812,MCFARLAND,ROSENTHALL,MCFARLAND,SHEILA,SHEILA,0,L,L,,...,1995-02-06,0,18957,18957,0,127461,,NaT,ROSENTHALL,9
9896,19802,ODUNSI,ODUNSI-CRAWL,ODUNSI,SHERRY,SHERRY,0,T,T,,...,2002-09-30,0,5855,20158,1,127507,,NaT,ODUNSICRAWL,5
3645,19834,OLIVARES,VERA,OLIVARES,GLORIA,GLORIA,0,,,,...,2010-04-16,0,7691,7691,0,109965,,NaT,VERA,6
10168,19926,ORTIZ-CLAUDIO,ORTIZ,ORTIZCLAUDIO,SUSIE,SUSIE,0,E,E,,...,1994-10-03,0,18551,18551,0,128223,,NaT,ORTIZ,7
3253,21180,ROLDAN,GRUBE,ROLDAN,EVELYN,EVELYN,0,,,,...,2005-09-26,0,11676,11676,0,108422,,NaT,GRUBE,6
8119,21237,ROSADO,RODRIGUEZ,ROSADO,NYVEA,NYVEA,0,I,I,,...,2007-07-09,0,15539,15539,0,121977,,NaT,RODRIGUEZ,7
8750,21355,RUTLEDGE,RUTLEDGE-WILLIS,RUTLEDGE,REGINA,REGINA,0,D,D,,...,1999-03-08,0,6485,6485,0,123883,,NaT,RUTLEDGEWILLIS,6


In [55]:
len(_)

36

In [56]:
all_merge[rel_columns][all_merge["first_name_comp"]>=3]

,id,last name,last_name,last_name_key_oos,first name,first_name,first_name_comp,middle initial,middle_initial,middle_initial2,...,org_hire_date,employment_date_comp,badge number,current_star,badge_number_comp,UID,suffix_name,resignation_date,last_name_key_cpdp,last_name_comp
3184,18448,MARSH,MARSH,MARSH,ERIN E M,ERIN,5,,E,M,...,1991-11-18,0,14440,14440,0,108158,,2017-01-15,MARSH,0
4012,19101,MIZERA,MIZERA,MIZERA,JACKIE,JACQUELINE,5,M,M,,...,2005-09-26,0,5037,20259,1,111109,,NaT,MIZERA,0
3882,22655,TO,TO,TO,HO,HOYIN,3,Y,T,,...,2015-08-31,0,NaN,19426,-1,110774,,NaT,TO,0
8684,13598,CERDA,CERDA,CERDA,RAUL JR,RAUL,3,,,,...,2000-12-18,0,15848,15848,0,123584,JR,NaT,CERDA,0
5507,13562,CAUINIAN,CAUINIAN,CAUINIAN,JUAN A G,JUAN,4,,A,G,...,1999-11-29,0,7275,7275,0,115998,,NaT,CAUINIAN,0
10825,12729,BEY,BRADLEY,BEY,NACALA,VALENA,4,M,M,,...,1989-12-26,0,17810,17810,0,129956,,NaT,BRADLEY,4


In [57]:
all_merge[rel_columns][all_merge["gender_comp"]!=0]

,id,last name,last_name,last_name_key_oos,first name,first_name,first_name_comp,middle initial,middle_initial,middle_initial2,...,org_hire_date,employment_date_comp,badge number,current_star,badge_number_comp,UID,suffix_name,resignation_date,last_name_key_cpdp,last_name_comp


In [58]:
all_merge[rel_columns][all_merge["race_comp"]!=0]

,id,last name,last_name,last_name_key_oos,first name,first_name,first_name_comp,middle initial,middle_initial,middle_initial2,...,org_hire_date,employment_date_comp,badge number,current_star,badge_number_comp,UID,suffix_name,resignation_date,last_name_key_cpdp,last_name_comp
7988,19662,NUNEZ,NUNEZ,NUNEZ,NICHOLAS,NICHOLAS,0,E,E,,...,2010-09-01,0,13672,13672,0,121704,,NaT,NUNEZ,0
3299,21208,ROMERO,ROMERO,ROMERO,FERNANDO,FERNANDO,0,,,,...,2015-02-23,0,13223,13223,0,108530,,NaT,ROMERO,0
5095,16159,GUZMAN,GUZMAN,GUZMAN,JONATHAN,JONATHAN,0,B,B,,...,2014-09-29,0,15766,15766,0,114930,,NaT,GUZMAN,0


In [59]:
all_merge[rel_columns][all_merge["birth_year_comp"]>0]

,id,last name,last_name,last_name_key_oos,first name,first_name,first_name_comp,middle initial,middle_initial,middle_initial2,...,org_hire_date,employment_date_comp,badge number,current_star,badge_number_comp,UID,suffix_name,resignation_date,last_name_key_cpdp,last_name_comp
4176,17358,KELLY,KELLY,KELLY,JAMES,JAMES,0,M,M,,...,1993-12-13,0,21121,21121,0,111735,,NaT,KELLY,0
11592,17106,JOHNSON,JOHNSON,JOHNSON,WARREN,WARREN,0,D,D,,...,2004-09-27,0,17442,17442,0,132081,,NaT,JOHNSON,0
3685,16914,JACOBS,JACOBS,JACOBS,GREGORY,GREGORY,0,S,S,,...,1999-07-12,0,17440,17440,0,110070,,NaT,JACOBS,0
10430,16765,HUGHES,HUGHES,HUGHES,THOMAS,THOMAS,0,J,J,,...,2000-08-14,0,18449,18449,0,129000,,NaT,HUGHES,0
4873,16705,HOWARD,HOWARD,HOWARD,JOHN,JOHN,0,D,D,,...,2013-07-15,0,15524,15524,0,113929,,NaT,HOWARD,0
5198,18124,LOPEZ,LOPEZ,LOPEZ,JOSE,JOSE,0,L,L,,...,2006-05-01,0,11943,11943,0,115080,,NaT,LOPEZ,0
8905,18171,LOSIK,LOSIK,LOSIK,RICHARD,RICHARD,0,F,F,,...,1998-06-29,0,991,991,0,124358,,NaT,LOSIK,0
7474,18244,LYNCH,LYNCH,LYNCH,MICHAEL,MICHAEL,0,F,F,,...,1991-06-19,0,21278,21278,0,120602,,NaT,LYNCH,0
7498,18511,MARTINEZ,MARTINEZ,MARTINEZ,MICHAEL,MICHAEL,0,A,A,,...,1998-08-31,0,19064,19064,0,120640,,NaT,MARTINEZ,0
5666,18587,MATTSON,MATTSON,MATTSON,KARL,KARL,0,M,M,,...,2002-12-02,0,3615,3615,0,116332,,NaT,MATTSON,0


In [60]:
all_merge[rel_columns][all_merge["middle_initial_comp"]>0]

,id,last name,last_name,last_name_key_oos,first name,first_name,first_name_comp,middle initial,middle_initial,middle_initial2,...,org_hire_date,employment_date_comp,badge number,current_star,badge_number_comp,UID,suffix_name,resignation_date,last_name_key_cpdp,last_name_comp
11629,17737,LACKO,LACKO,LACKO,ELIAS,ELIAS,0,,D,,...,2001-08-27,0,11575,11575,0,132119,,NaT,LACKO,0
2583,17606,KOUNAVIS,KOUNAVIS,KOUNAVIS,DENNIS,DENNIS,0,,N,,...,1996-11-04,0,19175,19175,0,106210,,NaT,KOUNAVIS,0
8302,17396,KENNING,KENNING,KENNING,PATRICK,PATRICK,0,,J,,...,2007-07-30,0,8302,8302,0,122474,,NaT,KENNING,0
3840,17338,KEELER,KEELER,KEELER,HERBERT,HERBERT,0,,M,,...,1999-05-10,0,3729,3729,0,110606,,NaT,KEELER,0
1107,17256,KANE,KANE,KANE,BRIAN,BRIAN,0,,P,,...,2001-02-05,0,1815,1815,0,102617,,NaT,KANE,0
10721,17173,JONES,JONES,JONES,TOMMIE,TOMMIE,0,,W,,...,1999-12-13,0,6268,6268,0,129763,,NaT,JONES,0
10612,17172,JONES,JONES,JONES,TIMOTHY,TIMOTHY,0,,P,,...,2006-02-21,0,12266,12266,0,129589,,NaT,JONES,0
10262,17170,JONES,JONES,JONES,TERESA,TERESA,0,,A,,...,2007-06-04,0,8704,8704,0,128405,,NaT,JONES,0
11589,17060,JOHNSON,JOHNSON,JOHNSON,EDWARD,EDWARD,0,,J,,...,2003-08-25,0,9539,9539,0,132078,,NaT,JOHNSON,0
9941,16983,JARZAB,JARZAB,JARZAB,SOPHIE,SOPHIE,0,,T,,...,2006-08-28,0,7741,7741,0,127621,,NaT,JARZAB,0


In [61]:
all_merge[rel_columns][all_merge["rank_comp"]==0]

,id,last name,last_name,last_name_key_oos,first name,first_name,first_name_comp,middle initial,middle_initial,middle_initial2,...,org_hire_date,employment_date_comp,badge number,current_star,badge_number_comp,UID,suffix_name,resignation_date,last_name_key_cpdp,last_name_comp
9124,17511,KLICH,KLICH,KLICH,ROBERT,ROBERT,0,H,H,,...,1982-08-30,0,5,5,0,125255,,NaT,KLICH,0
9130,17750,LAJEWSKI,LAJEWSKI,LAJEWSKI,ROBERT,ROBERT,0,J,J,,...,1985-12-16,0,23,23,0,125287,,2016-12-15,LAJEWSKI,0
1847,17749,LAIDLAW,LAIDLAW,LAIDLAW,CRAIG,CRAIG,0,T,T,,...,2013-03-05,0,17695,17695,0,104530,,NaT,LAIDLAW,0
3829,17748,LAI,LAI,LAI,HENRY,HENRY,0,H,H,,...,2010-09-01,0,16486,10025,1,110545,,NaT,LAI,0
4893,17747,LAHORI,LAHORI,LAHORI,JOHN,JOHN,0,K,K,,...,1990-07-30,0,9852,9852,0,114072,,2016-03-15,LAHORI,0
9607,17746,LAGUNAS,LAGUNAS,LAGUNAS,SAMUEL,SAMUEL,0,,,,...,2005-09-26,0,19202,19202,0,126965,,NaT,LAGUNAS,0
173,17745,LAGUNAS,LAGUNAS,LAGUNAS,ALEJANDRO,ALEJANDRO,0,,,,...,2002-10-28,0,9916,9916,0,100411,,NaT,LAGUNAS,0
608,17744,LAFATA,LAFATA,LAFATA,ANTHONY,ANTHONY,0,B,B,,...,1995-06-05,0,7176,7176,0,101394,,NaT,LAFATA,0
6835,17743,LADUZINSKY,LADUZINSKY,LADUZINSKY,MARK,MARK,0,J,J,,...,1995-01-03,0,17231,17231,0,119168,,NaT,LADUZINSKY,0
9525,17742,LADD,LADD,LADD,RYAN,RYAN,0,M,M,,...,2001-05-29,0,11878,11878,0,126778,,NaT,LADD,0


In [62]:
all_merge[rel_columns][all_merge["employment_date_comp"]!=0]

,id,last name,last_name,last_name_key_oos,first name,first_name,first_name_comp,middle initial,middle_initial,middle_initial2,...,org_hire_date,employment_date_comp,badge number,current_star,badge_number_comp,UID,suffix_name,resignation_date,last_name_key_cpdp,last_name_comp


In [63]:
all_merge[rel_columns][all_merge["badge_number_comp"]>0]

,id,last name,last_name,last_name_key_oos,first name,first_name,first_name_comp,middle initial,middle_initial,middle_initial2,...,org_hire_date,employment_date_comp,badge number,current_star,badge_number_comp,UID,suffix_name,resignation_date,last_name_key_cpdp,last_name_comp
3829,17748,LAI,LAI,LAI,HENRY,HENRY,0,H,H,,...,2010-09-01,0,16486,10025,1,110545,,NaT,LAI,0
3855,17730,LABOY JR,LABOY,LABOY,HERNAN,HERNAN,0,,,,...,1990-03-26,0,10920,19177,1,110669,JR,2017-05-15,LABOY,0
4189,17726,LABBE,LABBE,LABBE,JAMES,JAMES,0,C,C,,...,2001-07-30,0,1179,624,1,111778,,NaT,LABBE,0
7458,17691,KUMIGA,KUMIGA,KUMIGA,MICHAEL,MICHAEL,0,J,J,,...,2000-02-28,0,15998,2426,1,120562,,NaT,KUMIGA,0
2365,17688,KUMIEGA,KUMIEGA,KUMIEGA,DAVID,DAVID,0,J,J,,...,1998-08-31,0,21290,963,1,105676,,NaT,KUMIEGA,0
4186,17668,KUBIK,KUBIK,KUBIK,JAMES,JAMES,0,,,,...,1990-03-26,0,1397,757,1,111771,,NaT,KUBIK,0
5665,17655,KRUGER,KRUGER,KRUGER,KARL,KARL,0,D,D,,...,2006-05-22,0,17501,1505,1,116329,,NaT,KRUGER,0
7455,17654,KRUEGER,KRUEGER,KRUEGER,MICHAEL,MICHAEL,0,P,P,,...,2001-08-27,0,4529,20653,1,120554,,NaT,KRUEGER,0
6104,17639,KRIZKA,KRIZKA,KRIZKA,KRISTOPHE,KRISTOPHE,0,K,K,,...,2003-10-27,0,15972,20560,1,117251,,NaT,KRIZKA,0
9672,17630,KRAVITZ,KRAVITZ,KRAVITZ,SCOTT,SCOTT,0,D,D,,...,2006-08-28,0,2252,1046,1,127130,,NaT,KRAVITZ,0


# save

In [64]:
all_merge.to_csv("1_out_all_merged.csv", index=False)

In [65]:
all_matched_df.to_csv("1_out_merge_key.csv", index=False)